    Lab Assignment 1 Exercise, Problem 2:  A Rank Ordered List of the Ten Largest Counties By Population
    in Washington State in Calendar Year 2016 

    Objective:  Print out a ranked list, in descending order, of the ten largest counties based on the
    2016 population estimates from the Washington State Office of Fiscal Management.  The data file:
    saep_bg10.shp has the population estimates for the Census Block Groups in each County.

In [14]:
#   Step 1:  Setup the work environment to correspond to the ArcGIS Desktop in the CSDE Lab, Sociology
#   Department, 122 Raitt Hall, UW

import matplotlib

import sys
sys.path.append( 'C:\\Program Files (x86)\\ArcGIS\\Desktop10.3\\bin' )
sys.path.append( 'C:\\Program Files (x86)\\ArcGIS\\Desktop10.3\\arcpy' )
sys.path.append( 'C:\\Program Files (x86)\\ArcGIS\\Desktop10.3\\ArcToolbox\\Scripts' )

import arcpy
arcpy.env.workspace = 'U:\\Geog458\\Lab1'
arcpy.env.overwriteOutput = True


#   Step 2:  Make a backup copy of the WA State Census Block Group shapefile identified by its filename:
#   saep_bg10.shp so that the original file stays intact and continues as it was created by the WA State
#   OFM; as a result, this backup copy called under a new filename:  saep_bg10_copy.shp is now the working
#   file for this project.

arcpy.management.CopyFeatures(
    'U:\\Geog458\\Lab1\\saep_bg10.shp',
    'U:\\Geog458\\Lab1\\saep_bg10_copy.shp')


#   Step 3:  Setup a County_FIPS to County_Name dictionary where the key is the County FIPS code and the
#   corresponding value is the County Name, a character string; retrieve this information from the input
#   file:  WashingtonFIPS.dbf  and put it all in the dictionary:  County_Name_Dict.

input_feature_class = 'U:\\Geog458\\Lab1\\WashingtonFIPS.dbf'
input_cursor = arcpy.da.SearchCursor( input_feature_class, ["CountyName", "CountyFIPS"] )

County_Name_Dict = dict()
for row in input_cursor:
    County_Name = row[0]
    County_FIPS = row[1]
    County_Name_Dict[County_FIPS] = County_Name
del input_cursor


#   Step 4:  Setup a County_FIPS_List from the County_Name_Dict dictionary

County_FIPS_List = []
for (County_FIPS, County_Name) in County_Name_Dict.items():
    County_FIPS_List = County_FIPS_List + [County_FIPS]
Sorted_County_FIPS_List = sorted(County_FIPS_List)


#   Step 5:  For every County, select all the records in the shapefile that match the given County
#   FIPS code.  Put those records in a temporary file and process the individual records in that
#   file.  For each county, the County_Population count is put into the County_Population_List
#   where the initial value is the County_FIPS code and the corresponding value is the County_Population
#   count.  Use the SQL Select statement and Where clause.  Source:  Zandbergen, Chapter 7.3:  Using SQL
#   in Python, Page 146, Paragraph 1.  

County_Population_List = []

for County_FIPS in Sorted_County_FIPS_List:
    
    input_feature_class = 'U:\\Geog458\\Lab1\\saep_bg10_copy.shp'
    temporary_output_feature_class = 'U:\\Geog458\\Lab1\\saep_bg10_output.shp'
    fieldname = 'CountyFIPS'
    variable = County_FIPS
    where_clause = '{} = {}'.format(arcpy.AddFieldDelimiters(input_feature_class, fieldname), variable)
    arcpy.Select_analysis(input_feature_class,
                          temporary_output_feature_class,
                          where_clause)
    
    temporary_input_feature_class = 'U:\\Geog458\\Lab1\\saep_bg10_output.shp'
    with arcpy.da.SearchCursor(temporary_input_feature_class, ["CountyFIPS", "POP2016"] ) as temporary_input_cursor:
        
        County_Population = 0
        for row in temporary_input_cursor:
            County_FIPS_Code = row[0]
            County_Block_Group_Population = row[1]
            County_Population = County_Population + County_Block_Group_Population
        del temporary_input_cursor
        County_Population_List = County_Population_List + [(County_FIPS, County_Population)]
    
    
#   Step 6:  Sort the County Population list by the count rather than the code; the code is
#   the County FIPS; the count is the County Population which is the second item, in position 1,
#   in the operator:  itemgetter(1).

from operator import itemgetter
Counties_Sorted_By_Population = \
    sorted(County_Population_List, key = itemgetter(1), reverse = True)  
    

#   Step 7:  Make an output list that has the County FIPS, County Name, and County Population
#   for each county in the State; print the content of the output list, in this case, the
#   first ten lines of data.

Output_List = []
for County_Record in range(0, len(Counties_Sorted_By_Population), 1):
    County_FIPS = Counties_Sorted_By_Population[County_Record][0]
    County_Population = Counties_Sorted_By_Population[County_Record][1]
    Output_List = Output_List + [(County_FIPS, County_Population)]

for Output_Line in range(0, 10, 1):
    County_FIPS = Output_List[Output_Line][0]
    County_Name = County_Name_Dict[County_FIPS]
    County_Population = Output_List[Output_Line][1]
    print "%3d  %-12s %10.0f " % (County_FIPS, County_Name, County_Population)





    


    
    



    






 33  King            2105100 
 53  Pierce           844490 
 61  Snohomish        772860 
 63  Spokane          492530 
 11  Clark            461010 
 67  Thurston         272690 
 35  Kitsap           262590 
 77  Yakima           250900 
 73  Whatcom          212540 
  5  Benton           190500 


    Bob Klug, Arcpy Lab Assignment 1, Geog 458:  Advanced Digital Geographies Using ArcGIS Python Scripting

In [ ]:
    Lab Assignment 1 Exercise, Problem 1:  Extraction and GeoJSON Conversion of the County Shapefiles

In [ ]:
    Objective:  For each county in the State, create a GeoJSON file whose filename is the county name
    and whose contents are the polygons for the block groups in that county.  Each file should be named
    after a specific county and contain only the block groups from that county.

In [ ]:
#   Step 1:  Setup the work environment to correspond to the ArcGIS Desktop in the CSDE Lab, Sociology
#   Department, 122 Raitt Hall, UW; then reconfigure the work environment to the Geography Department
#   Lab, 401 Smith Hall, UW at the appropriate time because the GeoJSON file conversion packages are
#   only available in that one location.  So the code development is a two part process.

import matplotlib

import sys
sys.path.append( 'C:\\Program Files (x86)\\ArcGIS\\Desktop10.4\\bin' )
sys.path.append( 'C:\\Program Files (x86)\\ArcGIS\\Desktop10.4\\arcpy' )
sys.path.append( 'C:\\Program Files (x86)\\ArcGIS\\Desktop10.4\\ArcToolbox\\Scripts' )

import arcpy
arcpy.env.workspace = 'U:\\Geog458\\Lab1'
arcpy.env.overwriteOutput = True

from subprocess import call
import os
os.environ["GOAL_DATA"] = "C:\\OSGeo4W\\share\\gdal"
os.environ["GOAL_DRIVER_PATH"] = "C:\\OSGeo4W\\bin\\gdalplugins"
os.environ["PROJ_LIB"] = "C:\\OSGeo4W\\share\\proj"
os.environ["PATH"] = \
"C:\\OSGeo4W\\bin;"+os.environ["PATH"]+";C:\\OSGeo4W\\apps\\msys\\bin;C:\\OSGeo4W\\apps\\Python27\\Scripts"


#   Step 2:  Make a backup copy of the WA State Census Block Group shapefile identified by its filename:
#   saep_bg10.shp so that the original file stays intact and continues as it was created by the WA State
#   OFM; as a result, this backup copy called under a new filename:  saep_bg10_copy.shp is now the working
#   file for this project.

arcpy.management.CopyFeatures(
    'U:\\Geog458\\Lab1\\saep_bg10.shp',
    'U:\\Geog458\\Lab1\\saep_bg10_copy.shp')


#   Step 3:  Setup a County_FIPS to County_Name dictionary where the key is the County FIPS code and the
#   corresponding value is the County Name, a character string; retrieve this information from the input
#   file:  WashingtonFIPS.dbf  and put it all in the dictionary:  County_Name_Dict.

input_feature_class = 'U:\\Geog458\\Lab1\\WashingtonFIPS.dbf'
input_cursor = arcpy.da.SearchCursor( input_feature_class, ["CountyName", "CountyFIPS"] )

County_Name_Dict = dict()
for row in input_cursor:
    County_Name = row[0]
    County_FIPS = row[1]
    County_Name_Dict[County_FIPS] = County_Name
del input_cursor


#   Step 4:  Setup a County_FIPS_List from the County_Name_Dict dictionary

County_FIPS_List = []
for (County_FIPS, County_Name) in County_Name_Dict.items():
    County_FIPS_List = County_FIPS_List + [County_FIPS]
Sorted_County_FIPS_List = sorted(County_FIPS_List)


#   Step 5:  For every County, select all the records in the shapefile that match the given County
#   FIPS code.  Put those records in a temporary file and split out only the shape field from these
#   records; put these shape field only records into a separate file where the fieldname contains the
#   county name.    

for County_FIPS in Sorted_County_FIPS_List:
    
#   Step 5A:  Select County records from WA State Block Group Shapefile  
    
    input_feature_class = 'U:\\Geog458\\Lab1\\saep_bg10_copy.shp'
    temporary_output_filename = 'temporary_output_filename.shp'
    temporary_output_feature_class = 'U:\\Geog458\\Lab1\\' + temporary_output_filename
    fieldname = 'CountyFIPS'
    variable = County_FIPS
    where_clause = '{} = {}'.format(arcpy.AddFieldDelimiters(input_feature_class, fieldname), variable)
    arcpy.Select_analysis(input_feature_class,
                          temporary_output_feature_class,
                          where_clause)
    
#   Step 5B:  Create field name list
    
    temporary_input_feature_class = temporary_output_feature_class
    field_name_list = arcpy.ListFields(temporary_input_feature_class)
    
#   Step 5C:  Create delete field name list
      
    delete_field_name_list =[]
    for field_name in field_name_list:
        field_name_list_element = str(field_name.name)
        delete_field_name_list = delete_field_name_list + [field_name_list_element]
    while delete_field_name_list.count("Shape") > 0:
        delete_field_name_list.remove("Shape")
    while delete_field_name_list.count("FID") > 0:
        delete_field_name_list.remove("FID")
    while delete_field_name_list.count("COUNTYFP10") > 0:
        delete_field_name_list.remove("COUNTYFP10")

    
#   Step 5D:  Delete unneeded fields from the County shapefile.  File now has only the
#   the system required fields:  FID, Shape, and one additional field per ArcGIS standards.
    
    arcpy.DeleteField_management(temporary_input_feature_class, delete_field_name_list)

#   Step 5E:  Copy temporary file into the output file; output file has the filename:
#   countyname.shp
        
    temporary_output_feature_class = temporary_input_feature_class
    output_filename = County_Name_Dict[County_FIPS] + '.shp'
    output_feature_class = 'U:\\Geog458\\Lab1\\' + output_filename
    arcpy.management.CopyFeatures(
        temporary_output_feature_class,
        output_feature_class)
    temporary_input_feature_class = output_feature_class
    temporary_output_feature_class = 'C:\\Users\\rhklug\\' + output_filename
    arcpy.management.CopyFeatures(
        temporary_input_feature_class,
        temporary_output_feature_class)
    print output_filename
    print output_feature_class
    print temporary_output_feature_class
    
#   Step 5F:  Execute GeoJSON file conversion

    arcpy.env.workspace = 'C:\\Users\\rhklug'
    GeoJSON_Output_File_Name = County_Name_Dict[County_FIPS]
    ArcGIS_Input_File_Name = County_Name_Dict[County_FIPS]
    call(['C:\\OSGeo4W\\bin\\ogr2ogr',
         '-f','GeoJSON','-t_srs','WGS84',
         '-s_srs','EPSG:26913',
         'C:\\Users\\rhklug\\GeoJSON_Output_File_Name.geojson',
         'C:\\Users\\rhklug\\ArcGIS_Input_File_Name.shp'])
    print GeoJSON_Output_File_Name

print "File Conversion Pricess Complete"
    
    

In [ ]:
    Bob Klug, Arcpy Lab Assignment 1, Geog 458:  Advanced Digital Geographies Using ArcGIS Python Scripting